In [ ]:
#importing basic and necessary python libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Importing the input data
df = pd.read_excel("Question/Input.xlsx")

,URL_ID,URL
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...
4,Netclan20241021,https://insights.blackcoffer.com/development-o...
...,...,...
142,Netclan20241159,https://insights.blackcoffer.com/population-an...
143,Netclan20241160,https://insights.blackcoffer.com/google-lsa-ap...
144,Netclan20241161,https://insights.blackcoffer.com/healthcare-da...
145,Netclan20241162,https://insights.blackcoffer.com/budget-sales-...


# Data Extraction

In [3]:
#Function to extract text from URL using beautiful soup library

def Data_Extraction(URL_ID,URL):

    # importing URL
    import requests
    webpage=requests.get(URL).text

    #using beautiful soup library for web scraping
    from bs4 import BeautifulSoup
    soup=BeautifulSoup(webpage,'lxml')
    article_text=soup.find_all('div',class_='td-post-content tagdiv-type')

    #Removing HTML tags from the text extracted
    import re 
    def remove_html_tags(text):
        pattern=re.compile('<.*?>')
        text = pattern.sub(r' ',str(text))
        return text
    final_text = remove_html_tags(article_text)

    #Saving Extracted text into a text file with URL_ID as its filename
    with open(f'Extracted Data/{URL_ID}.text','w') as file:
        for word in final_text.split():
            file.write(word+"\n")

    return final_text

In [4]:
# creating article_text column in the data frame as well along with extracting data and storing it in external file as well
df['article_text']=None

for i in range(0,df.shape[0]):
    URL_ID = df['URL_ID'][i]
    URL = df['URL'][i]
    
    df['article_text'][i]=''.join(Data_Extraction(URL_ID,URL))

In [7]:
#Determining encoding type of text files for opening file in wight way before extracting stopwords
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        detector = chardet.universaldetector.UniversalDetector()
        for line in file:
            detector.feed(line)
            if detector.done:
                break
        detector.close()
    encoding = detector.result['encoding']
    print(f'The encoding of the {file_path} is: {encoding}')

file_path = ['Question/StopWords/StopWords_Auditor.txt',
             'Question/StopWords/StopWords_Currencies.txt',
             'Question/StopWords/StopWords_DatesandNumbers.txt',
             'Question/StopWords/StopWords_Generic.txt',
             'Question/StopWords/StopWords_GenericLong.txt',
             'Question/StopWords/StopWords_Geographic.txt',
             'Question/StopWords/StopWords_Names.txt'
            ]

for i in file_path:
    detect_encoding(i)

The encoding of the Question/StopWords/StopWords_Auditor.txt is: ascii
The encoding of the Question/StopWords/StopWords_Currencies.txt is: ISO-8859-1
The encoding of the Question/StopWords/StopWords_DatesandNumbers.txt is: ascii
The encoding of the Question/StopWords/StopWords_Generic.txt is: ascii
The encoding of the Question/StopWords/StopWords_GenericLong.txt is: ascii
The encoding of the Question/StopWords/StopWords_Geographic.txt is: ascii
The encoding of the Question/StopWords/StopWords_Names.txt is: ascii


In [8]:
# As the files are in two types of encoding, we are forming two functions for extracting stopwords, one for each encoding type

def load_stopwords_ascii(file_path):
    with open(file_path, 'r',encoding='ascii') as file:
        stopwords=set(file.read().splitlines())  #Adding to set for fast lookup
    return stopwords

def load_stopwords_ISO(file_path):
    with open(file_path, 'r',encoding='ISO-8859-1') as file:
        stopwords=set(file.read().splitlines())  #Adding to set for fast lookup
    return stopwords

In [9]:
#function to extract stopwords 
def extract_stopwords():

    file_paths_ISO=['Question/StopWords/StopWords_Currencies.txt']
    file_paths_ascii=['Question/StopWords/StopWords_Auditor.txt',
                      'Question/StopWords/StopWords_DatesandNumbers.txt',
                      'Question/StopWords/StopWords_Generic.txt',
                      'Question/StopWords/StopWords_GenericLong.txt',
                      'Question/StopWords/StopWords_Geographic.txt',
                      'Question/StopWords/StopWords_Names.txt'
                     ]

    stopwords=set()
    for i in file_paths_ISO:
        stopwords=stopwords.union(load_stopwords_ISO(i))
    
    for i in file_paths_ascii:
        stopwords=stopwords.union(load_stopwords_ascii(i))

    # lowercasing all the stopwords 
    lowercase_stopwords = {word.lower() for word in stopwords}

    print("Total stopwords extracted from files :",len(lowercase_stopwords))

    return lowercase_stopwords


#extracting stopwords
stopwords=set(extract_stopwords())

Total stopwords extracted from files : 12768


In [10]:
# Function to remove stopwords from a text
def remove_stopwords(text, stopwords):
    filtered_text=[]

    #tokenizing each word in the text using NLTK
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text)  
    
    # removing stopwords from tokenized text
    for word in text_words:
        if word not in stopwords:
            filtered_text.append(word)
    return filtered_text

In [11]:
# forming a column for text without stopwords
df['text_without_stopwords']=df['article_text'].str.lower()

for i in range(0,df.shape[0]):
    df['text_without_stopwords'][i]=' '.join(remove_stopwords(df['text_without_stopwords'][i], stopwords))

<h3>1. Sentiment Analysis</h3>

In [12]:
#Determining encoding type of text files for extracting sentiment words
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        detector = chardet.universaldetector.UniversalDetector()
        for line in file:
            detector.feed(line)
            if detector.done:
                break
        detector.close()
    encoding = detector.result['encoding']
    print(f'The encoding of the {file_path} is: {encoding}')

file_path = ['Question/MasterDictionary/negative-words.txt',
             'Question/MasterDictionary/positive-words.txt'
            ]

for i in file_path:
    detect_encoding(i)

The encoding of the Question/MasterDictionary/negative-words.txt is: ISO-8859-1
The encoding of the Question/MasterDictionary/positive-words.txt is: ascii


In [13]:
#Extracting Positive and negative words from the files present in master dictionary and lowercasing them
with open('Question/MasterDictionary/positive-words.txt', 'r',encoding='ascii') as file:
        positive_words=set(file.read().splitlines()) 
positive_words = {word.lower() for word in positive_words}

with open('Question/MasterDictionary/negative-words.txt', 'r',encoding='ISO-8859-1') as file:
        negative_words=set(file.read().splitlines())  
negative_words = {word.lower() for word in negative_words}

In [14]:
#functions to remove URLs and punctuations

import re
def remove_url(text):
    pattern=re.compile(r'https?://\S+\www\.\S+')
    return pattern.sub(r'',text)

import string
exclude=string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [15]:
#for effective NLP application for counting positive and negative words, removing URLs and punctuations
df['preprocessed_text']=None
for i in range(0,df.shape[0]):
    df['preprocessed_text'][i] = remove_url(df['text_without_stopwords'][i])
    df['preprocessed_text'][i] = remove_punc(df['text_without_stopwords'][i])

In [16]:
# functions to get positive and negative score from a text
def count_positive_score(text, words):
    count=0
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text)  # Tokenize using nltk
    
    for word in text_words:
        if word in words:
            count=count+1
            
    return count

def count_negative_score(text, words):
    count=0
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text)  # Tokenize using nltk
    
    for word in text_words:
        if word in words:
            count=count-1

    count = count*-1
            
    return count

In [17]:
# finding positive and negative score of the article text extracted
df['positive_score']=None
df['negative_score']=None

for i in range(0,df.shape[0]):
    df['positive_score'][i]=count_positive_score(df['preprocessed_text'][i],positive_words)
    df['negative_score'][i]=count_negative_score(df['preprocessed_text'][i],negative_words)

In [18]:
# function to find find polarity score of the article text extracted
def calc_polarity_score(positive_score,negative_score):
    polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    return polarity_score

In [19]:
#finding polarity score of the article text
df['polarity_score']=None
for i in range(0,df.shape[0]):
    df['polarity_score'][i]=calc_polarity_score(df['positive_score'][i],df['negative_score'][i])

In [20]:
# calculating total words after cleaning for subjectivity score
def total_words_after_cleaning(text):
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text)
    return len(text_words)

#finding total words after cleaning in article text
df['total_words_after_cleaning']=None
for i in range(0,df.shape[0]):
    df['total_words_after_cleaning'][i]=total_words_after_cleaning(df['preprocessed_text'][i])

In [21]:
#finding subjectivity score of the article text
def subjectivity_score(positive,negative,tot_words):
    sub_score = (positive + negative)/ (tot_words + 0.000001)
    return sub_score
    
df['subjectivity_score']=None
for i in range(0,df.shape[0]):
    df['subjectivity_score'][i]=subjectivity_score(df['positive_score'][i],df['negative_score'][i],df['total_words_after_cleaning'][i])

<h3>2. Analysis of Readability</h3>

In [23]:
# function to find number of words and sentences in a text
def num_of_words(text):
    text_words=remove_punc(text)
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text)
    return len(text_words)

def num_of_sent(text):
    from nltk.tokenize import sent_tokenize
    text_sent = sent_tokenize(text)
    return len(text_sent)

In [24]:
# finding total number of sentences and words in the article text
df['total_num_of_words']=None
df['total_num_of_sent']=None

for i in range(0,df.shape[0]):
    df['total_num_of_words'][i]=num_of_words(df['article_text'][i])
    df['total_num_of_sent'][i]=num_of_sent(df['article_text'][i])

In [25]:
# finding Average Sentence Length 
df['average_sentence_length']=None
for i in range(0,df.shape[0]):
    df['average_sentence_length'][i]=df['total_num_of_words'][i]/df['total_num_of_sent'][i]

In [26]:
#function to find number of complex words in a text
def complex_word_count(text):
    
    #removing punctuation from text and tokenizing it as well
    text_words=remove_punc(text)
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text)

    #using rules for syllables as provided for finding number of complex words
    vowels="AEIOUaeiou"
    num_complex_words=0
    for word in text_words:
        count = 0
        if not (word.endswith('es') or word.endswith('ed')):
            for char in word:
                if char in vowels:
                    count=count+1

        if count>2:
            num_complex_words=num_complex_words+1

    return num_complex_words

In [27]:
# finding number of complex words in article_text
df['num_complex_words']=None
for i in range(0,df.shape[0]):
    df['num_complex_words'][i]=complex_word_count(df['article_text'][i])

In [28]:
# finding percentage of complex words in article_text
df['percentage_of_complex_words']=None
for i in range(0,df.shape[0]):
    df['percentage_of_complex_words'][i]=df['num_complex_words'][i]/df['total_num_of_words'][i]

In [29]:
# finding fog index of the article text
df['fog_index']=None
for i in range(0,df.shape[0]):
    df['fog_index'][i]=0.4*(df['average_sentence_length'][i]+df['percentage_of_complex_words'][i])

In [30]:
# finding average number of words per sentence
df['avg_num_words_per_sent']=None
for i in range(0,df.shape[0]):
    df['avg_num_words_per_sent'][i]=df['total_num_of_words'][i]/df['total_num_of_sent'][i]

In [31]:
# function to remove punctuation from a text
import string
exclude=string.punctuationu
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

# function to remove stopwords present in nltk from a text 
def remove_stopwords_by_nltk(text):
    filtered_text=[]
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text)  # Tokenize using nltk

    from nltk.corpus import stopwords
    for word in text_words:
        if word not in stopwords.words('english'):
            filtered_text.append(word)
    return filtered_text

# function to find word count in a text
def word_count(text):
    text=remove_punc(text)
    text=remove_stopwords_by_nltk(text)
    return len(text)

In [32]:
# finding word count in the article
df['word_count']=None
for i in range(0,df.shape[0]):
    df['word_count'][i]=word_count(df['article_text'][i])

In [33]:
# function to found Syllable count per word
def syllable_count_per_word(text):

    # removing punctuation and tokenizing
    text_words=remove_punc(text)
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text_words)

    #using rules given to find number of syllables in each word
    vowels="AEIOUaeiou"
    syllable_count=[]
    for word in text_words:
        count = 0
        if not (word.endswith('es') or word.endswith('ed')):
            for char in word:
                if char in vowels:
                    count=count+1
                    
        if count>=1:
            syllable_count.append(count)
        else:
            syllable_count.append("No Syllables")

    return syllable_count

In [34]:
# finding syllable count per word in each article
df['syllable_count_per_word']=None
for i in range(0,df.shape[0]):
    df['syllable_count_per_word'][i]=syllable_count_per_word(df['article_text'][i])

In [35]:
# function to find count of personal pronouns
def count_personal_pronouns(text):

    # special care taken for US country name 
    text=text.replace('US','')
    text=text.lower()

    # using regex for checking for personal pronouns
    import re
    pattern = r'\b(I|we|my|ours|us)\b'  # \b ensures whole word matching

    # Use re.findall() to find all occurrences
    matches = re.findall(pattern, text, flags=re.IGNORECASE)

    return len(matches

In [40]:
# finding count of personal pronouns in each article
df['count_personal_pronouns']=None
for i in range(0,df.shape[0]):
    df['count_personal_pronouns'][i]=count_personal_pronouns(df['article_text'][i])

In [36]:
# function to find total characters in an article
def total_char_each_word(text):
    text_words=remove_punc(text)
    from nltk.tokenize import word_tokenize
    text_words = word_tokenize(text)

    count=0
    for word in text_words:
        for char in word:
            count=count+1

    return count

In [37]:
# finding average words length of an article
df['average_word_length']=None
for i in range(0,df.shape[0]):
    df['average_word_length'][i]=total_char_each_word(df['article_text'][i])/df['total_num_of_words'][i]

In [46]:
# forming dataframe as given in the output data structure

#Renaming columns
df=df.rename(columns={'positive_score': 'POSITIVE SCORE',
                     'negative_score': 'NEGATIVE SCORE',
                     'polarity_score': 'POLARITY SCORE',
                     'subjectivity_score' : 'SUBJECTIVITY SCORE',
                     'average_sentence_length' : 'AVG SENTENCE LENGTH',
                     'percentage_of_complex_words' : 'PERCENTAGE OF COMPLEX WORDS',
                     'fog_index' : 'FOG INDEX',
                     'avg_num_words_per_sent' : 'AVG NUMBER OF WORDS PER SENTENCE',
                     'num_complex_words' : 'COMPLEX WORD COUNT',
                     'word_count': 'WORD COUNT',
                     'syllable_count_per_word' : 'SYLLABLE PER WORD',
                     'count_personal_pronouns' : 'PERSONAL PRONOUNS',
                     'average_word_length' : 'AVG WORD LENGTH'
                    })

#Selecting Required columns
output_df = (df[['URL_ID',
                'URL',
                'POSITIVE SCORE', 
                'NEGATIVE SCORE',
                'POLARITY SCORE', 
                'SUBJECTIVITY SCORE',
                'AVG SENTENCE LENGTH',
                'PERCENTAGE OF COMPLEX WORDS',
                'FOG INDEX',
                'AVG NUMBER OF WORDS PER SENTENCE',
                'COMPLEX WORD COUNT', 
                'WORD COUNT', 
                'SYLLABLE PER WORD',
                'PERSONAL PRONOUNS', 
                'AVG WORD LENGTH']])

#Final data output
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,5,1,0.666667,0.033898,51.833333,0.221865,20.822079,51.833333,69,214,"[2, 3, 2, 1, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, ...",3,5.315113
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,8,8,0.0,0.038186,27.888889,0.158367,11.218902,27.888889,159,535,"[2, 3, 2, 1, 3, 3, 2, 1, 1, 2, 2, 1, 4, 2, 1, ...",7,4.424303
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,11,4,0.466667,0.04918,40.75,0.224949,16.38998,40.75,110,344,"[2, 3, 2, 1, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, ...",1,5.02863
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,22,11,0.333333,0.080882,24.777778,0.303438,10.032486,24.777778,203,458,"[2, 3, 2, 1, 3, 6, 1, 1, 1, 1, 1, 2, 2, 1, No ...",4,5.765321
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,9,2,0.636364,0.023013,19.209302,0.217918,7.770888,19.209302,180,519,"[4, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 3, 2, 2, 1, ...",1,4.939467


In [44]:
# Saving output to a csv file
output_df.to_csv("OUTPUT.csv")